# The random walk revisited

We are going to analyze the random walk on a $d$-dimensional lattice in $d=1,2,3$ with periodic boundary conditions. To do so we will use a `struct Lattice` that generates neighbors for lattices in arbitrary dimensions, defined in the `lattice.jl` file.

In [1]:
using PyPlot, SpecialFunctions, Revise
includet("lattice.jl")

Let's see an example for a 3-d cubic lattice of size $5\times 5 \times 5$

In [ ]:
mydims = (5,5,5)
L = Lattice(mydims) # note that I could have used Lattice(5,5,5)

To figure out what is going on, let's print the `LinearIndex`, the `CartesianIndex` and the neighbors' list for each site of the lattice. Note that the functions `lin2cart` is defined in `lattice.jl`. Note also the idiomatic syntax for dispatching on a function of more than one input using the syntax `Ref` on the 2nd argument of the function `lin2cart`.

In [ ]:
[vec(L.site) lin2cart.(L.site, Ref(L.dims)) L.neig]

## The Random Walk in arbitrary dimensions

Now we define a method for doing the random-walk. By default we start in the lattice centre.

In [ ]:
function randomwalk(L::Lattice,nsteps::Int)
    d = length(L.dims)
    traj = zeros(Int, nsteps+1)
    traj[1] = cart2lin(L.dims,L.dims .>> 1 .+ 1) #lattice centre initialization saved as first trajectory point / .>> = *0.5
    for i in 2:nsteps+1
        traj[i] = L.neig[traj[i-1]][rand(1:2d)]
    end
    traj
end

We generate 3 lattices di $d=1,2,3$ of size $10^6$, $1001\times1001$, and $101\times 101\times 101$ respectively. Then we generate `nwalker=10^6` trajectories.

In [ ]:
L1 = Lattice(1_000_001) # 1D lattice   
L2 = Lattice(1001,1001) # 2D lattice
L3 = Lattice(101,101,101) # 3D ... smaller lattice because 1001,1001,1001 is too big
nwalkers = 1_000_000
titer = 300
res1 = [randomwalk(L1,titer) for i in 1:nwalkers]; #diffusion in 1D
res2 = [randomwalk(L2,titer) for i in 1:nwalkers]; #diffusion in 2D
res3 = [randomwalk(L3,titer) for i in 1:nwalkers]; #diffusion in 3D

Plot a trajectory...

In [ ]:
idx_traj = 1
idxx = [tup[1] for tup in lin2cart.(res2[idx_traj], Ref(L2.dims))]
idxy = [tup[2] for tup in lin2cart.(res2[idx_traj], Ref(L2.dims))]
plot(idxx, idxy)
scatter(idxx[1], idxy[1], marker = "s", color = "r")
scatter(idxx[end],  idxy[end], marker = "o", color = "g")
xticks(collect(minimum(idxx):1:maximum(idxx)))
yticks(collect(minimum(idxy):1:maximum(idxy)))
grid(which= "major")
xlabel("x-coordinate")
ylabel("y-coordinate")


We now define a function to compute the average euclidean distance from the starting point of our trajectory. Note that to translate LinearIndices into the Cartesian representation, we use the `lin2cart` method defined in the file `lattice.jl`.

In [ ]:
function trajdiffusion(vtraj::Vector{Vector{Int}}, L::Lattice)
    ntraj = length(vtraj)
    ave = zeros(length(vtraj[1]))
    va = similar(ave)
    for t in 1:length(vtraj[1])
        media = 0.0
        mom2 = 0.0
        for i in 1:ntraj
            p0 = lin2cart(vtraj[i][1],L.dims)
            pt = lin2cart(vtraj[i][t],L.dims)
            dt = sqrt(sum((p0 .- pt).^2))
            media += dt
            mom2 += dt^2
        end
        ave[t] = media/ntraj
        va[t] = (mom2-media*media/ntraj)/ntraj
    end
    ave,va
end

In [ ]:
m1,e1 = trajdiffusion(res1,L1); # 2D statistics
m2,e2 = trajdiffusion(res2,L2); # 2D statistics
m3,e3 = trajdiffusion(res3,L3); # 3D statistics

## <center> RMSD and average (Euclidean) distance </center> 

Let us compute 

$$
RMSD(t) = \sqrt{\langle \vec R(t)^2  \rangle} 
$$

and

$$
D(t)= \langle |\vec R(t)| \rangle = \langle \sqrt{\vec R(t) \cdot \vec R(t)} \rangle
$$

and verify numerically the theoretical result for $D(t)$, and $RMSD(t)$.